# Variational Autoencoder for anomaly detection in Predictive Maintenance

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### Define path to dataset

In [0]:
load_data = pd.read_csv('train_modified.csv',sep=',')
#del load_data['Unnamed: 0.1']
#del load_data['Unnamed: 0']
load_data.to_csv('train.csv')
DATA_PATH = 'train.csv'

## Dataset Overview

In [0]:
df_base = pd.read_csv(DATA_PATH,sep=',',index_col=False)
df_base.head()
cols=df_base.columns
cols

Index(['Unnamed: 0', 'X097_DE_time', 'X097_FE_time'], dtype='object')

## Build Data Loader

In [0]:
def load_and_standardize_data(path):
    # read in from csv
    df = pd.read_csv(path, sep=',')
    # replace nan with 0
    df = df.fillna(0)
    df = df.values.reshape(-1, df.shape[1]).astype('float32')
    # randomly split
    X_train, X_test = train_test_split(df, test_size=0.3, random_state=42)
    # standardize values
    scaler = preprocessing.StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)   
    return X_train, X_test, scaler

In [0]:
from torch.utils.data import Dataset, DataLoader
class DataBuilder(Dataset):
    def __init__(self, path, train=True):
        self.X_train, self.X_test, self.standardizer = load_and_standardize_data(DATA_PATH)
        if train:
            self.x = torch.from_numpy(self.X_train)
            self.len=self.x.shape[0]
        else:
            self.x = torch.from_numpy(self.X_test)
            self.len=self.x.shape[0]
        del self.X_train
        del self.X_test 
    def __getitem__(self,index):      
        return self.x[index]
    def __len__(self):
        return self.len

In [0]:
traindata_set=DataBuilder(DATA_PATH, train=True)
testdata_set=DataBuilder(DATA_PATH, train=False)

trainloader=DataLoader(dataset=traindata_set,batch_size=256)
testloader=DataLoader(dataset=testdata_set,batch_size=256)

In [0]:
type(trainloader.dataset.x), type(testloader.dataset.x)

(torch.Tensor, torch.Tensor)

In [0]:
trainloader.dataset.x.shape, testloader.dataset.x.shape

(torch.Size([14378, 3]), torch.Size([6162, 3]))

In [0]:
trainloader.dataset.x

tensor([[-0.7310,  0.8153, -0.9797],
        [-1.2989,  1.5323, -1.4553],
        [-1.3574,  0.0120, -0.6427],
        ...,
        [-0.8181,  1.2616, -0.5904],
        [-1.5799,  0.1157,  0.0106],
        [ 0.9316, -1.6551,  0.8233]])

## Build model

In [0]:
class VariationalAutoencoder(nn.Module):
    def __init__(self,D_in,H=50,H2=12,latent_dim=2):
        
        #Encoder
        super(VariationalAutoencoder,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.lin_bn1 = nn.BatchNorm1d(num_features=H)
        self.linear2=nn.Linear(H,H2)
        self.lin_bn2 = nn.BatchNorm1d(num_features=H2)
        self.linear3=nn.Linear(H2,H2)
        self.lin_bn3 = nn.BatchNorm1d(num_features=H2)
        
        # Latent vectors mu and sigma
        self.fc1 = nn.Linear(H2, latent_dim)
        self.bn1 = nn.BatchNorm1d(num_features=latent_dim)
        self.fc21 = nn.Linear(latent_dim, latent_dim)
        self.fc22 = nn.Linear(latent_dim, latent_dim)

        # Sampling vector
        self.fc3 = nn.Linear(latent_dim, latent_dim)
        self.fc_bn3 = nn.BatchNorm1d(latent_dim)
        self.fc4 = nn.Linear(latent_dim, H2)
        self.fc_bn4 = nn.BatchNorm1d(H2)
        
        # Decoder
        self.linear4=nn.Linear(H2,H2)
        self.lin_bn4 = nn.BatchNorm1d(num_features=H2)
        self.linear5=nn.Linear(H2,H)
        self.lin_bn5 = nn.BatchNorm1d(num_features=H)
        self.linear6=nn.Linear(H,D_in)
        self.lin_bn6 = nn.BatchNorm1d(num_features=D_in)
        
        self.relu = nn.ReLU()
        
    def encode(self, x):
        lin1 = self.relu(self.lin_bn1(self.linear1(x)))
        lin2 = self.relu(self.lin_bn2(self.linear2(lin1)))
        lin3 = self.relu(self.lin_bn3(self.linear3(lin2)))

        fc1 = F.relu(self.bn1(self.fc1(lin3)))

        r1 = self.fc21(fc1)
        r2 = self.fc22(fc1)
        
        return r1, r2
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu
        
    def decode(self, z):
        fc3 = self.relu(self.fc_bn3(self.fc3(z)))
        fc4 = self.relu(self.fc_bn4(self.fc4(fc3)))

        lin4 = self.relu(self.lin_bn4(self.linear4(fc4)))
        lin5 = self.relu(self.lin_bn5(self.linear5(lin4)))
        return self.lin_bn6(self.linear6(lin5))
        
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [0]:
class customLoss(nn.Module):
    def __init__(self):
        super(customLoss, self).__init__()
        self.mse_loss = nn.MSELoss(reduction="sum")
    
    def forward(self, x_recon, x, mu, logvar):
        loss_MSE = self.mse_loss(x_recon, x)
        loss_KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return loss_MSE + loss_KLD

In [0]:
D_in = traindata_set.x.shape[1]
H = 50
H2 = 12
model = VariationalAutoencoder(D_in, H, H2).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [0]:
loss_mse = customLoss()

## Train Model

In [0]:
epochs = 200
log_interval = 50
val_losses = []
train_losses = []
test_losses = []

In [0]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_mse(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    if epoch % 50 == 0:        
        print('====> Epoch: {} Average training loss: {:.4f}'.format(
            epoch, train_loss / len(trainloader.dataset)))
        train_losses.append(train_loss / len(trainloader.dataset))

In [0]:
def test(epoch):
    with torch.no_grad():
        test_loss = 0
        for batch_idx, data in enumerate(testloader):
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)
            loss = loss_mse(recon_batch, data, mu, logvar)
            test_loss += loss.item()
            if epoch % 50 == 0:        
                print('====> Epoch: {} Average test loss: {:.4f}'.format(
                    epoch, test_loss / len(testloader.dataset)))
            test_losses.append(test_loss / len(testloader.dataset))

In [0]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

====> Epoch: 50 Average training loss: 2.6004
====> Epoch: 50 Average test loss: 0.1046
====> Epoch: 50 Average test loss: 0.2098
====> Epoch: 50 Average test loss: 0.3228
====> Epoch: 50 Average test loss: 0.4323
====> Epoch: 50 Average test loss: 0.5488
====> Epoch: 50 Average test loss: 0.6580
====> Epoch: 50 Average test loss: 0.7602
====> Epoch: 50 Average test loss: 0.8641
====> Epoch: 50 Average test loss: 0.9790
====> Epoch: 50 Average test loss: 1.0837
====> Epoch: 50 Average test loss: 1.1916
====> Epoch: 50 Average test loss: 1.2969
====> Epoch: 50 Average test loss: 1.4114
====> Epoch: 50 Average test loss: 1.5170
====> Epoch: 50 Average test loss: 1.6267
====> Epoch: 50 Average test loss: 1.7329
====> Epoch: 50 Average test loss: 1.8499
====> Epoch: 50 Average test loss: 1.9585
====> Epoch: 50 Average test loss: 2.0649
====> Epoch: 50 Average test loss: 2.1773
====> Epoch: 50 Average test loss: 2.2921
====> Epoch: 50 Average test loss: 2.3910
====> Epoch: 50 Average test l

Generate data from training dataset

In [0]:
with torch.no_grad():
    for batch_idx, data in enumerate(testloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)

In [0]:
scaler = trainloader.dataset.standardizer
recon_row = scaler.inverse_transform(recon_batch[0].cpu().numpy())
real_row = scaler.inverse_transform(testloader.dataset.x[0].cpu().numpy())


In [0]:
df = pd.DataFrame(np.stack((recon_row, real_row)), columns = cols)


In [0]:
sigma = torch.exp(logvar/2)

In [0]:
mu[1], sigma[1]

(tensor([-0.0014, -0.1264]), tensor([1.0033, 0.5766]))

In [0]:
mu.mean(axis=0), sigma.mean(axis=0)

(tensor([-0.0001, -0.0134]), tensor([1.0008, 0.5866]))

In [0]:
# sample z from q
no_samples = traindata_set.x.shape[0]
q = torch.distributions.Normal(mu.mean(axis=0), sigma.mean(axis=0))
z = q.rsample(sample_shape=torch.Size([no_samples]))

In [0]:
z.shape

torch.Size([14378, 2])

In [0]:
z[:5]

tensor([[ 0.2228,  0.0573],
        [-0.1123, -0.4240],
        [ 0.2120, -0.0476],
        [-0.4147,  0.4595],
        [-0.2707,  0.1881]])

In [0]:
with torch.no_grad():
    pred = model.decode(z).cpu().numpy()

In [0]:
pred[1]

array([ 0.0462416 , -0.49644354,  0.5208957 ], dtype=float32)

## Generate data from Variational Autoencoder

In [0]:
fake_data = scaler.inverse_transform(pred)
fake_data.shape


(14378, 3)

In [0]:
df_fake = pd.DataFrame(fake_data)


df_fake.columns=cols
df_fake.head(10)

In [0]:
load_datatest = pd.read_csv('test_modified.csv',sep=',')
#del load_datatest['Unnamed: 0.1']
#del load_datatest['Unnamed: 0']
load_datatest.to_csv('test.csv')

TEST_PATH = 'test.csv'
df_anomaly = pd.read_csv(TEST_PATH, sep=',')
cols_ano = df_anomaly.columns
anomalydata_set=DataBuilder(TEST_PATH,train=True)
anomalyloader=DataLoader(anomalydata_set,batch_size=256)

In [0]:
#generate from anomaly data
with torch.no_grad():
    for batch_idx, data in enumerate(anomalyloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)

scaler = anomalyloader.dataset.standardizer
recon_test_row = scaler.inverse_transform(recon_batch[0].cpu().numpy())
sigma = torch.exp(logvar/2)

no_samples_test = df_anomaly.shape[0]
q = torch.distributions.Normal(mu.mean(axis=0), sigma.mean(axis=0))
z = q.rsample(sample_shape=torch.Size([no_samples_test]))


In [0]:
with torch.no_grad():
    pred_test = model.decode(z).cpu().numpy()
fake_data_test = scaler.inverse_transform(pred_test)

df_fake_test = pd.DataFrame(fake_data_test)

df_fake_test.columns=cols_ano


In [0]:
#load data from inner fault
load_datatest = pd.read_csv('testir_modified.csv',sep=',')
#del load_datatest['Unnamed: 0.1']
#del load_datatest['Unnamed: 0']
load_datatest.to_csv('testir.csv')

TEST_PATH = 'testir.csv'
df_anomaly_ir = pd.read_csv(TEST_PATH, sep=',')
cols_ano = df_anomaly_ir.columns
anomalydata_set=DataBuilder(TEST_PATH,train=True)
anomalyloader=DataLoader(anomalydata_set,batch_size=256)

#generate from anomaly data
with torch.no_grad():
    for batch_idx, data in enumerate(anomalyloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)

scaler = anomalyloader.dataset.standardizer
recon_test_row = scaler.inverse_transform(recon_batch[0].cpu().numpy())
sigma = torch.exp(logvar/2)

no_samples_test = df_anomaly_ir.shape[0]
q = torch.distributions.Normal(mu.mean(axis=0), sigma.mean(axis=0))
z = q.rsample(sample_shape=torch.Size([no_samples_test]))

with torch.no_grad():
    pred_test = model.decode(z).cpu().numpy()
fake_data_test = scaler.inverse_transform(pred_test)

df_fake_test_ir = pd.DataFrame(fake_data_test)

df_fake_test_ir.columns=cols_ano


In [0]:
#load data from outer fault
load_datatest = pd.read_csv('testor_modified.csv',sep=',')
#del load_datatest['Unnamed: 0.1']
#del load_datatest['Unnamed: 0']
load_datatest.to_csv('testor.csv')

TEST_PATH = 'testor.csv'
df_anomaly_or = pd.read_csv(TEST_PATH, sep=',')
cols_ano = df_anomaly_or.columns
anomalydata_set=DataBuilder(TEST_PATH,train=True)
anomalyloader=DataLoader(anomalydata_set,batch_size=256)

#generate from anomaly data
with torch.no_grad():
    for batch_idx, data in enumerate(anomalyloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)

scaler = anomalyloader.dataset.standardizer
recon_test_row = scaler.inverse_transform(recon_batch[0].cpu().numpy())
sigma = torch.exp(logvar/2)

no_samples_test = df_anomaly_or.shape[0]
q = torch.distributions.Normal(mu.mean(axis=0), sigma.mean(axis=0))
z = q.rsample(sample_shape=torch.Size([no_samples_test]))

with torch.no_grad():
    pred_test = model.decode(z).cpu().numpy()
fake_data_test = scaler.inverse_transform(pred_test)

df_fake_test_or = pd.DataFrame(fake_data_test)

df_fake_test_or.columns=cols_ano


In [0]:
#reconstruction error train data
mse = ((df_base-df_fake)**2).mean(axis=1)


In [0]:
#reconstruction error ball data
mse_test = ((df_anomaly-df_fake_test)**2).mean(axis=1)


In [0]:
#reconstruction error inner data
mse_test_ir = ((df_anomaly_ir-df_fake_test_ir)**2).mean(axis=1)


In [0]:
#reconstruction error outer data
mse_test_or = ((df_anomaly_or-df_fake_test_or)**2).mean(axis=1)


In [0]:
#calculate anomaly threshold from reconstruction error train data
threshold = mse.max()


In [0]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(figsize=(10, 6), dpi=80)
plt.plot(mse[0:200],'go',label='normal')
plt.plot(mse_test[0:200],'bD',label='ball')
plt.plot(mse_test_ir[0:200],'r^',label='inner')
plt.plot(mse_test_or[0:200],'k*',label='outer')
plt.axhline(y=threshold,color='r', linestyle='-',label='threshold')
plt.legend(loc=4)
plt.xlabel("Observations")
plt.ylabel("Score")
plt.title('Anomaly Detection Results')
plt.savefig('AD.pdf')
plt.show()